In [121]:
import glob
import cv2
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(linewidth=np.inf)
import os
import tensorflow as tf
from tensorflow.keras import layers, models

# Binary Masks

In [58]:
images = glob.glob("./font/*/*/*.png"); # array of image file paths (features)

for file in images:
    # loading image as a grayscale
    image = cv2.imread(file,  cv2.IMREAD_GRAYSCALE);
    normalized_image = image / 255.0 # normalizing

    # handwriting extraction and obtaining a binary mask
    _, binary_mask = cv2.threshold(normalized_image, 0.7, 1, cv2.THRESH_BINARY_INV)
    binary_mask_path = f'{file[0:-4]}_binary_mask.png';
    #print(binary_mask_path)
    if not cv2.imwrite(binary_mask_path, (binary_mask * 255).astype(np.uint8)):
        raise Exception("Could not write image")


In [97]:
binary_masks = glob.glob("./font/*/*/*_binary_mask.png");

In [115]:
# Function to load and preprocess images
def load_and_preprocess_image(image_path):
    # Load the image
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=1)
    image = float(image) / 255.0
    return image

# Function to load and preprocess binary masks
def load_and_preprocess_mask(mask_path):
    # Load the binary mask
    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)
    return mask

In [118]:
image_dataset = tf.data.Dataset.from_tensor_slices(images);
mask_dataset = tf.data.Dataset.from_tensor_slices(binary_masks);

# Apply the preprocessing functions to the datasets
image_dataset = image_dataset.map(load_and_preprocess_image)
mask_dataset = mask_dataset.map(load_and_preprocess_mask)

combined_dataset = tf.data.Dataset.zip((image_dataset, mask_dataset))

# Print the first few elements of the combined dataset
for image, mask in combined_dataset.take(3):
    print("Image shape:", image.shape)
    print("Mask shape:", mask.shape)

Image shape: (234, 234, 1)
Mask shape: (234, 234, 1)
Image shape: (234, 234, 1)
Mask shape: (234, 234, 1)
Image shape: (234, 234, 1)
Mask shape: (234, 234, 1)


2024-01-22 16:39:18.807231: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype string and shape [2053]
	 [[{{node Placeholder/_3}}]]
